In [2]:
import pandas as pd
import folium

# 1. 파일 불러오기

In [3]:
df1 = pd.read_csv('./지진옥외대피소.csv',encoding = 'euckr')
df2 = pd.read_csv('./지진해일대피소.csv',encoding = 'euckr')

# 2. 지진 대피소
- 지진옥외와 지진해일대피소 합친 이유 설명

## 2-1. 지진옥외대피소 전처리
- 수용가능인원수 추가
- 위도 경도 값 변환

In [4]:
df1['수용가능인원수'] = round(df1['시설면적']/3.3).astype(int)

In [5]:
df1.loc[df1['위도']=='35.15.02','위도'] = '35.1502'
df1.loc[df1['경도'] =='128.37.08','경도'] = '128.3708'
df1.loc[df1['위도']=='35.15.14','위도'] = '35.1514'
df1.loc[df1['경도']=='128.36.28','경도'] = '128.3628'
df1.loc[df1['경도']=='35.481726, 126.481198','경도'] = '126.481198'

## 2-2. 지진옥외 / 지진해일 대피소 중복값 찾기

### 2-2-1. 지진옥외대피소

In [6]:
#### 지진옥외대피소 겹치는 값 확인 코드
df1[df1['상세주소'].duplicated(keep=False)]

,지역코드,시설일련번호,시도명,시군구명,시설명,상세주소,시설면적,경도,위도,수용가능인원수
83,4315000000,3,충청북도,제천시,금성초등학교 운동장,충청북도 제천시 금성면 구룡리 294-0,5953,128.16544612145643,37.06719875510558,1804
84,4315000000,49,충청북도,제천시,금성초등학교 운동장,충청북도 제천시 금성면 구룡리 294-0,5953,128.16548565149745,37.06723442591937,1804
324,4374500000,7,충청북도,증평군,형석중학교 운동장,충청북도 증평군 증평읍 미암리 603,7753,127.59314496391403,36.79279093756006,2349
325,4374500000,9,충청북도,증평군,형석고등학교 운동장,충청북도 증평군 증평읍 미암리 603,8400,127.59314496391403,36.79279093756006,2545
439,1159000000,19,서울특별시,동작구,영등포고등학교 운동장,서울특별시 동작구 대방동 13-2,5241,126.93383427187928,37.507988916849975,1588
...,...,...,...,...,...,...,...,...,...,...
10400,1129000000,35,서울특별시,성북구,길음중학교 운동장,서울특별시 성북구 길음동 635-7,2062,127.01838463022357,37.61363429459085,625
10404,1129000000,56,서울특별시,성북구,서울대학교 사범대학 부설 중학교 운동장,서울특별시 성북구 종암동 7-10,18452,127.03896757562556,37.5958036172146,5592
10405,1129000000,57,서울특별시,성북구,서울대학교 사범대학 부설 고등학교 운동장,서울특별시 성북구 종암동 7-10,12736,127.03781065509901,37.592726843326155,3859
10668,4719000000,137,경상북도,구미시,오상고등학교 운동장,경상북도 구미시 장천면 상장리 700,8500,128.49672620498637,36.12386556888312,2576


In [7]:
#### 지진옥외대피소 겹치는 값의 시설명이 각각 다른 곳인지 확인 코드
# - 4곳이 같은 것을 볼 수 있다.
df1[df1['상세주소'].duplicated(keep=False)]['시설명'].value_counts()

금성초등학교 운동장      2
빛가온중학교 운동장      2
자경2어린이공원        2
경기항공고등학교 운동장    2
미성어린이공원         1
               ..
서울예술고등학교 운동장    1
후창공원            1
광희중학교 운동장       1
무학중학교 운동장       1
오상중학교 운동장       1
Name: 시설명, Length: 215, dtype: int64

In [8]:
lis_duplicate = ['빛가온중학교 운동장','자경2어린이공원','경기항공고등학교 운동장']

In [9]:
# 지진옥외대피소에서 겹치는 4곳 지우기.
df1 = df1.drop(index = df1[df1['시설명'] == '금성초등학교 운동장'][:1].index)
for g in lis_duplicate:
    df1 = df1.drop(index = df1[df1['시설명'] == g][:1].index)
df1.reset_index(inplace =  True)

### 2-2-2. 지진해일대피소

In [10]:
#### 지진해일대피소 겹치는 값 확인 코드
df2[df2['주소'].duplicated(keep = False)]

,일련번호,시도명,시군구명,대피지구명,대피장소명,주소,경도,위도,수용가능인원수,해변으로부터거리,대피소 분류명,내진적용여부,해발높이
14,622,경상북도,울진군,후포지구1,마을뒷산고지대,경상북도 울진군 후포면 후포리 37,129.464291,36.688566,200,120,공터,NaN,20
15,623,경상북도,울진군,후포지구1,마을뒷산 고지대,경상북도 울진군 후포면 후포리 37,129.464335,36.688388,200,100,공터,NaN,19
283,268,강원도,양양군,남애지구,남애리 야산 1,강원도 양양군 현남면 남애리 1,128.786052,37.945723,200,300,공터,NaN,30
284,269,강원도,양양군,남애지구,남애리 야산 2,강원도 양양군 현남면 남애리 1,128.785876,37.941682,200,200,공터,NaN,25
285,270,강원도,양양군,남애지구,남애리 야산 3,강원도 양양군 현남면 남애리 1,128.786258,37.945462,200,300,공터,NaN,25


In [11]:
# 지진해일대피소 겹치는 곳을 지도를 통해 확인 한 결과 다른 곳임을 알게 되었다.(drop 대상에서 제외.)
df = df2[df2['주소'] == '강원도 양양군 현남면 남애리 1']

In [12]:
# pip install folium

In [13]:
import folium
map = folium.Map(location=[36, 127], zoom_start=7)
for i in df.index:
    lat = df.loc[i, '위도']
    lon = df.loc[i, '경도']
    marker = folium.Marker([lat,lon]).add_to(map)
map

### 2-2-3. 지진해일 / 지진옥외 주소 합치기

In [14]:
df1_test = df1[['상세주소']]
df2_test = df2[['주소']]
df2_test = df2_test.rename({'주소':'상세주소'},axis =1)
df_address = pd.concat([df1_test,df2_test],axis = 0,ignore_index=True)

# df1_test 이랑 df2_test 겹치면 df2_test 결과 가져오려고 keep='first'
df_address_tot = df_address[df_address.duplicated(keep = 'first')]

In [15]:
df1.shape

(10872, 11)

In [16]:
# drop할 대상이 아닌 지진해일대피소 중복값 확인
df2[df2['주소'].duplicated(keep = 'first')]

,일련번호,시도명,시군구명,대피지구명,대피장소명,주소,경도,위도,수용가능인원수,해변으로부터거리,대피소 분류명,내진적용여부,해발높이
15,623,경상북도,울진군,후포지구1,마을뒷산 고지대,경상북도 울진군 후포면 후포리 37,129.464335,36.688388,200,100,공터,NaN,19
284,269,강원도,양양군,남애지구,남애리 야산 2,강원도 양양군 현남면 남애리 1,128.785876,37.941682,200,200,공터,NaN,25
285,270,강원도,양양군,남애지구,남애리 야산 3,강원도 양양군 현남면 남애리 1,128.786258,37.945462,200,300,공터,NaN,25


In [17]:
# df1 / df2 각각의 겹치는 값들은 그대로 두기 위해 (서로 다르기 때문에)
# duplicated list에서 지운다.
df_address_tot = df_address_tot.drop(index = [10887,11156,11157])

### 2-2-4. 지진옥외 / 지진해일 대피소 겹치는값 list 보관

In [18]:
## 지진옥외 / 지진해일 대피소 겹치는값.
## 10872는 df1의 row값
duplicated_list = df_address_tot[df_address_tot.index > 10872].index

## 2-3. 지진해일 / 지진옥외 대피소 데이터프레임 합치기

In [19]:
# 합친 후 인구 변화도 함께 볼 것이기 때문에 지진옥외 대피소와 지진해일 대피소의 '시군구명'이 겹치는 부분 변경
df2.loc[df2['시군구명'] == '포항시남구','시군구명'] = '포항시 남구'

In [20]:
df2.rename({'대피장소명':'시설명','주소':'상세주소'}, axis=1,inplace = True)

In [21]:
#필요한 칼럼만 남기기
df1 = df1[['시도명','시군구명','시설명','상세주소','경도','위도','수용가능인원수']]
df2 = df2[['시도명','시군구명','시설명','상세주소','경도','위도','수용가능인원수']]
df_total = pd.concat([df1,df2],axis = 0 , ignore_index= True)

## 2-4. 지진해일 / 지진옥외 대피소 중복값 제거

In [22]:
# 중복값 제거
df_total = df_total.drop(index = duplicated_list, axis = 0)
df_total.reset_index(drop='index', inplace = True)

In [23]:
#중복값 없는 것을 확인.
df_total[df_total.duplicated()]

,시도명,시군구명,시설명,상세주소,경도,위도,수용가능인원수


## 2-5. 데이터 저장

In [24]:
# df_total.to_csv('지진대피소 종합본.csv',encoding='euckr',index=False)

# 3. 포항시 세분화 과정

## 3-1. 포항으로 나눔

In [25]:
df = pd.read_csv('지진대피소 종합본.csv',encoding='euckr')

In [26]:
pohang = df[df['시군구명'].str.contains('포항')]

In [27]:
pohang

,시도명,시군구명,시설명,상세주소,경도,위도,수용가능인원수
589,경상북도,포항시,양학중학교 운동장,경상북도 포항시 북구 득량동 205-6,129.342419,36.032992,1030
590,경상북도,포항시,양학생활체육운동장,경상북도 포항시 북구 득량동 219-0,129.340831,36.036154,1970
591,경상북도,포항시,양학초등학교 운동장,경상북도 포항시 북구 학잠동 191,129.344584,36.031967,1515
600,경상북도,포항시,선린요양병원 주차장,경상북도 포항시 북구 대신동 74-1,129.367309,36.046569,1061
601,경상북도,포항시,(구)포항중앙초등학교 운동장,경상북도 포항시 북구 동빈1가 90-0,129.368515,36.043652,1606
...,...,...,...,...,...,...,...
11258,경상북도,포항시북구,환호 여중 앞,경상북도 포항시 북구 환호동 69,129.400759,36.071086,370
11260,경상북도,포항시북구,화진1리 버스정류장 옆,경상북도 포항시 북구 송라면 화진리 261-1,129.382398,36.236579,210
11261,경상북도,포항시북구,화진2리 화진비치펜션 앞,경상북도 포항시 북구 송라면 화진리 180-1,129.381916,36.239312,208
11262,경상북도,포항시북구,포항해양과학고 앞,경상북도 포항시 북구 환호동 2-8,129.406166,36.074059,200


In [28]:
# pohang.to_csv('포항시 대피소 현황.csv',index = False)

## 3-2. 포항의 유동인구 / 총인구수 대비 수용인원 고려

In [29]:
popo = pd.read_csv('포항_유동인구리스트.csv')

In [30]:
lis =['흥해읍',
 '장량동',
 '중앙동',
 '오천읍',
 '죽도동',
 '대이동',
 '우창동',
 '연일읍',
 '제철동',
 '상대동',
 '효곡동',
 '용흥동',
 '해도동',
 '두호동',
 '청하면',
 '청림동',
 '동해면',
 '송도동',
 '구룡포읍',
 '송라면',
 '기계면',
 '대송면',
 '환여동',
 '장기면',
 '양학동',
 '호미곶면',
 '죽장면',
 '신광면',
 '기북면']

In [31]:
data = []
for s in lis:
    temp = []
    path = f'./포항시/{s}대피소.csv'
    # print(path)
    df1 = pd.read_csv(path,encoding = 'euckr')
    ret = df1['수용가능인원수'].sum()
    temp.append(s)
    temp.append(ret)
    data.append(temp)

In [32]:
th = pd.DataFrame(data,columns = ['행정동명','수용인원수합'])

In [33]:
### 유동인원 + 총인구수 나눈 값

In [34]:
ro = pd.read_csv('주민등록인구및세대현황_월간.csv',encoding = 'euckr')
ro = ro[['행정구역','2019년08월_총인구수']]
yi = ro['행정구역'].str.split().to_list()

In [35]:
temp = []
for i in range(len(yi)):
    temp.append(yi[i][-1])

In [36]:
fi = pd.DataFrame(temp)
ro = pd.concat([ro,fi],axis = 1, ignore_index=False)
ro = ro.drop(index = [0,1,16,23],axis = 0 ).reset_index(drop = 'index')

In [37]:
a = '구룡포읍 연일읍 오천읍 대송면 동해면 장기면 호미곶면 상대동 해도동 송도동 청림동 제철동 효곡동 대이동 흥해읍 신광면 청하면 송라면 기계면 죽장면 기북면 중앙동 양학동 죽도동 용흥동 우창동 두호동 장량동 환여동'
lis = a.split()
li = pd.DataFrame(lis,columns = ['호우'])
ro = pd.concat([ro,li], axis = 1 , ignore_index= False)

In [38]:
roro = ro[['2019년08월_총인구수','호우']]
roro = roro.rename({'호우':'행정동명'},axis = 1)

In [39]:
#유동인구수 대비 수용인원비율 생성
popo = pd.merge(popo,th,on='행정동명')
popo['유동인구수 대비 수용인원비율'] = popo['수용인원수합'] / popo['유동인구수'] *100
popo = popo.sort_values('유동인구수 대비 수용인원비율',ascending = False)
popo_total = pd.merge(popo,roro,on='행정동명')

In [40]:
popo_total['2019년08월_총인구수']= popo_total['2019년08월_총인구수'].str.replace(',','')
popo_total['2019년08월_총인구수'] = popo_total['2019년08월_총인구수'].astype(int)
popo_total['총인구수 대비 수용인원비율'] = popo_total['수용인원수합']/popo_total['2019년08월_총인구수']
popo_total['총인구수 대비 수용인원비율'] = popo_total['총인구수 대비 수용인원비율']*100
popo_total.sort_values('총인구수 대비 수용인원비율',ascending = False)

,구,행정동명,유동인구수,수용인원수합,유동인구수 대비 수용인원비율,2019년08월_총인구수,총인구수 대비 수용인원비율
1,남구,장기면,4053.387,13958,344.353993,4291,325.285481
8,북구,송라면,5901.258,6944,117.669826,2641,262.930708
5,남구,호미곶면,3357.581,5056,150.584602,1986,254.582075
0,북구,신광면,1660.226,5765,347.241882,2882,200.034698
4,북구,죽장면,2421.097,4068,168.023008,2174,187.120515
11,남구,대송면,5869.581,6155,104.862681,3689,166.847384
28,남구,제철동,21918.613,5151,23.500575,3104,165.947165
2,북구,환여동,5821.419,15780,271.067930,10000,157.800000
17,북구,청하면,10661.613,7336,68.807600,4919,149.136003
6,남구,구룡포읍,7378.871,10902,147.746180,7975,136.702194


In [41]:
popo_total.head()

,구,행정동명,유동인구수,수용인원수합,유동인구수 대비 수용인원비율,2019년08월_총인구수,총인구수 대비 수용인원비율
0,북구,신광면,1660.226,5765,347.241882,2882,200.034698
1,남구,장기면,4053.387,13958,344.353993,4291,325.285481
2,북구,환여동,5821.419,15780,271.067930,10000,157.800000
3,북구,기북면,648.323,1338,206.378611,1311,102.059497
4,북구,죽장면,2421.097,4068,168.023008,2174,187.120515


In [42]:
# popo.to_csv('포항 유동인구수 대비 수용인원비율.csv',index = False)

# 4. 포항시 대피소 시설 확인 작업

In [43]:
pohang =pd.read_csv('포항시 대피소 현황.csv')

In [44]:
pohang

,시도명,시군구명,시설명,상세주소,경도,위도,수용가능인원수
0,경상북도,포항시,양학중학교 운동장,경상북도 포항시 북구 득량동 205-6,129.342419,36.032992,2805
1,경상북도,포항시,양학생활체육운동장,경상북도 포항시 북구 득량동 219-0,129.340831,36.036154,5362
2,경상북도,포항시,양학초등학교 운동장,경상북도 포항시 북구 학잠동 191,129.344584,36.031967,4125
3,경상북도,포항시,선린요양병원 주차장,경상북도 포항시 북구 대신동 74-1,129.367309,36.046569,2888
4,경상북도,포항시,(구)포항중앙초등학교 운동장,경상북도 포항시 북구 동빈1가 90-0,129.368515,36.043652,4372
...,...,...,...,...,...,...,...
492,경상북도,포항시북구,환호 여중 앞,경상북도 포항시 북구 환호동 69,129.400759,36.071086,370
493,경상북도,포항시북구,화진1리 버스정류장 옆,경상북도 포항시 북구 송라면 화진리 261-1,129.382398,36.236579,210
494,경상북도,포항시북구,화진2리 화진비치펜션 앞,경상북도 포항시 북구 송라면 화진리 180-1,129.381916,36.239312,208
495,경상북도,포항시북구,포항해양과학고 앞,경상북도 포항시 북구 환호동 2-8,129.406166,36.074059,200


In [45]:
pohang.shape

(497, 7)

## 4-1. 시설명 value_counts

In [46]:
pohang2 =pd.read_csv('포항시 대피소 현황.csv')

In [47]:
pohang2['시설명'].str.split()

0            [양학중학교, 운동장]
1             [양학생활체육운동장]
2           [양학초등학교, 운동장]
3           [선린요양병원, 주차장]
4      [(구)포항중앙초등학교, 운동장]
              ...        
492           [환호, 여중, 앞]
493      [화진1리, 버스정류장, 옆]
494     [화진2리, 화진비치펜션, 앞]
495          [포항해양과학고, 앞]
496          [해안로511번길10]
Name: 시설명, Length: 497, dtype: object

In [48]:
te = pohang2['시설명'].to_list()

In [49]:
temp=[]
for i in te:
    temp+=i.split()

In [50]:
# 대피소 시설명이 겹친는 것을 알 수 있다.
pd.DataFrame(temp).value_counts().head(10)

주변       220
마을회관     206
운동장      111
앞         51
공터        21
옆         10
주차장        7
입구         7
경로당        7
버스승강장      7
dtype: int64

In [51]:
pohang2[pohang2['시설명'].str.contains('주변')]

,시도명,시군구명,시설명,상세주소,경도,위도,수용가능인원수
47,경상북도,포항시,덕장1리 마을회관 주변,경상북도 포항시 북구 흥해읍 덕장리 1025-17,129.334396,36.134192,165
48,경상북도,포항시,용전2리 마을회관 주변,경상북도 포항시 북구 흥해읍 용전리 250-2,129.342417,36.125855,165
49,경상북도,포항시,용전1리 마을회관 주변,경상북도 포항시 북구 흥해읍 용전리 797-21,129.329730,36.120748,165
53,경상북도,포항시,용곡리 마을회관 주변,경상북도 포항시 북구 흥해읍 용곡리 367-0,129.309805,36.122156,165
54,경상북도,포항시,양백1리 마을회관 주변,경상북도 포항시 북구 흥해읍 양백리 477-0,129.320755,36.125979,165
...,...,...,...,...,...,...,...
397,경상북도,포항시,눌태1리 마을회관 주변,경상북도 포항시 남구 구룡포읍 눌태리 225-0,129.538705,35.992794,165
407,경상북도,포항시,구평1리 마을회관 주변,경상북도 포항시 남구 구룡포읍 구평리 177-3,129.534054,35.942076,165
408,경상북도,포항시,장길리 마을회관 주변,경상북도 포항시 남구 구룡포읍 장길리 159-1,129.544058,35.953307,165
416,경상북도,포항시 남구,구만리 389-1 주변,경상북도 포항시 남구 호미곶면 구만리 389-1,129.550808,36.081356,185


### 4-1-1. 초중고등학교 / 공터

In [52]:
element = pohang[pohang['시설명'].str.contains('초등학교')]

In [53]:
middle = pohang[pohang['시설명'].str.contains('중학교')]

In [54]:
high = pohang[pohang['시설명'].str.contains('고등학교')]

In [55]:
high = high.drop(index=[365,395])

In [56]:
high

,시도명,시군구명,시설명,상세주소,경도,위도,수용가능인원수
5,경상북도,포항시,포항고등학교 운동장,경상북도 포항시 북구 학산동 288,129.371406,36.056372,9900
7,경상북도,포항시,포항여자고등학교 운동장,경상북도 포항시 북구 학산동 243,129.364874,36.050231,4125
21,경상북도,포항시,두호고등학교 운동장,경상북도 포항시 북구 두호동 1106-0,129.373251,36.062231,3300
23,경상북도,포항시,유성여자고등학교 운동장,경상북도 포항시 북구 우현동 205-2,129.360402,36.057321,2888
29,경상북도,포항시,경북과학고등학교 운동장,경상북도 포항시 북구 용흥동 418-1,129.352624,36.047091,2475
32,경상북도,포항시,세화고등학교 운동장,경상북도 포항시 북구 우현동 318-3,129.352384,36.053914,2888
33,경상북도,포항시,대동중고등학교 운동장,경상북도 포항시 북구 우현동 201-6,129.356779,36.058239,7920
36,경상북도,포항시,동지여자고등학교 운동장,경상북도 포항시 북구 용흥동 511-0,129.346384,36.037993,4620
40,경상북도,포항시,포항여자전자고등학교 운동장,경상북도 포항시 북구 용흥동 430-0,129.348970,36.050094,6352
71,경상북도,포항시,장성고등학교 운동장,경상북도 포항시 북구 장성동 1498-1,129.385180,36.085809,3465


In [57]:
pohang = pohang.drop(index = element.index)

In [58]:
pohang = pohang.drop(index = middle.index)

In [59]:
pohang = pohang.drop(index = high.index)

In [60]:
park = pohang[pohang['시설명'].str.contains('공원')]

In [61]:
element.shape , middle.shape, high.shape, park.shape

((71, 7), (24, 7), (21, 7), (28, 7))

In [62]:

pohang = pohang.drop(index = park.index)
pohang =pohang.reset_index(drop='index')

In [63]:
#나눠도 초교가 있다
pohang[pohang['시설명'].str.contains('초')]

,시도명,시군구명,시설명,상세주소,경도,위도,수용가능인원수
39,경상북도,포항시,초곡리 마을회관 주변,경상북도 포항시 북구 흥해읍 초곡리 1010-16,129.319440,36.086020,165
272,경상북도,포항시,(구)구남분교장(구룡포초교) 운동장,경상북도 포항시 남구 구룡포읍 구평리 739-1,129.536917,35.949315,3548
336,경상북도,포항시북구,월포초교 뒷뜰앞,경상북도 포항시 북구 청하면 용두리 481-2,129.367358,36.203407,506


In [64]:
pohang[pohang['시설명'].str.contains('중')]

,시도명,시군구명,시설명,상세주소,경도,위도,수용가능인원수
91,경상북도,포항시,중산1리 마을회관 주변,경상북도 포항시 북구 송라면 중산리 494-0,129.328536,36.236827,165
136,경상북도,포항시,중산2리 마을회관 주변,경상북도 포항시 북구 송라면 중산리 267-3,129.329406,36.239583,165
165,경상북도,포항시,중명2리 마을회관 주변,경상북도 포항시 남구 연일읍 중명리 631-2,129.315119,35.991243,165
166,경상북도,포항시,중명1리 마을회관 주변,경상북도 포항시 남구 연일읍 중명리 841-22,129.311714,35.996254,165
167,경상북도,포항시,중단1리 마을회관 주변,경상북도 포항시 남구 연일읍 중단리 536-1,129.325565,35.991500,165
168,경상북도,포항시,중단2리 마을회관 주변,경상북도 포항시 남구 연일읍 중단리 381-0,129.323685,35.990730,165
233,경상북도,포항시,중흥리 마을회관 주변,경상북도 포항시 남구 동해면 중흥리 225-0,129.504396,36.018492,165
348,경상북도,포항시북구,환호 여중 앞,경상북도 포항시 북구 환호동 69,129.400759,36.071086,370


In [65]:
pohang[pohang['시설명'].str.contains('고')]

,시도명,시군구명,시설명,상세주소,경도,위도,수용가능인원수
54,경상북도,포항시,고현1리 마을회관,경상북도 포항시 북구 청하면 고현리 402-0,129.329972,36.164418,165
55,경상북도,포항시,고현2리 마을회관,경상북도 포항시 북구 청하면 고현리 395-3,129.329881,36.166074,165
83,경상북도,포항시,고지리 마을쉼터 주변,경상북도 포항시 북구 기계면 고지리 496-0,129.178087,36.073219,165
139,경상북도,포항시,고지2리 마을회관 주변,경상북도 포항시 북구 기계면 고지리 45-0,129.188218,36.071159,165
263,경상북도,포항시,하정1리 선박출입대행신고소 주변,경상북도 포항시 남구 구룡포읍 하정리 261-1,129.547671,35.964895,165
346,경상북도,포항시북구,칠포1리 고지대,경상북도 포항시 북구 흥해읍 칠포리 54-4,129.396806,36.145619,70
351,경상북도,포항시북구,포항해양과학고 앞,경상북도 포항시 북구 환호동 2-8,129.406166,36.074059,200


### 4-1-2. 마을회관 공터 주차장

In [66]:
parking = pohang[pohang['시설명'].str.contains('주차장')]

In [67]:
town = pohang[pohang['시설명'].str.contains('마을회관')]

In [68]:
wide_place = pohang[pohang['시설명'].str.contains('공터')]

In [69]:
pohang = pohang.drop(index = parking.index)


In [70]:
pohang = pohang.drop(index = town.index)


In [71]:
wide_place = wide_place.drop(index = [51,331])

In [72]:
pohang = pohang.drop(index = wide_place.index)

In [73]:
parking

,시도명,시군구명,시설명,상세주소,경도,위도,수용가능인원수
1,경상북도,포항시,선린요양병원 주차장,경상북도 포항시 북구 대신동 74-1,129.367309,36.046569,2888
40,경상북도,포항시,농산물 도매시장 주차장,경상북도 포항시 북구 흥해읍 학천리 4-0,129.335629,36.080110,16500
90,경상북도,포항시,보경사 주차장,경상북도 포항시 북구 송라면 중산리 544-32,129.320535,36.243821,8250
140,경상북도,포항시,새마을운동기념관 주차장,경상북도 포항시 북구 기계면 문성리 298-1,129.195311,36.065083,1072
196,경상북도,포항시,양포항 주차장,경상북도 포항시 남구 장기면 양포리 9-32,129.519877,35.881513,1568
200,경상북도,포항시,석곡도서관 주차장,경상북도 포항시 남구 동해면 도구리 737-8,129.437695,35.990664,1650
257,경상북도,포항시,송도공영주차장,경상북도 포항시 남구 송도동 430-1,129.371486,36.034959,2888
258,경상북도,포항시,포항운하관 공영주차장,경상북도 포항시 남구 송도동 222-0,129.377326,36.023473,5775
260,경상북도,포항시,상가번영회 주차장,경상북도 포항시 남구 송도동 254-612,129.377999,36.036843,2062
262,경상북도,포항시,성모병원 야외주차장,경상북도 포항시 남구 대잠동 733-7,129.339822,36.014229,2888


In [74]:
town

,시도명,시군구명,시설명,상세주소,경도,위도,수용가능인원수
6,경상북도,포항시,덕장1리 마을회관 주변,경상북도 포항시 북구 흥해읍 덕장리 1025-17,129.334396,36.134192,165
7,경상북도,포항시,용전2리 마을회관 주변,경상북도 포항시 북구 흥해읍 용전리 250-2,129.342417,36.125855,165
8,경상북도,포항시,용전1리 마을회관 주변,경상북도 포항시 북구 흥해읍 용전리 797-21,129.329730,36.120748,165
9,경상북도,포항시,용곡리 마을회관 주변,경상북도 포항시 북구 흥해읍 용곡리 367-0,129.309805,36.122156,165
10,경상북도,포항시,양백1리 마을회관 주변,경상북도 포항시 북구 흥해읍 양백리 477-0,129.320755,36.125979,165
...,...,...,...,...,...,...,...
267,경상북도,포항시,눌태1리 마을회관 주변,경상북도 포항시 남구 구룡포읍 눌태리 225-0,129.538705,35.992794,165
273,경상북도,포항시,구평1리 마을회관 주변,경상북도 포항시 남구 구룡포읍 구평리 177-3,129.534054,35.942076,165
274,경상북도,포항시,장길리 마을회관 주변,경상북도 포항시 남구 구룡포읍 장길리 159-1,129.544058,35.953307,165
327,경상북도,포항시 남구,흥환리 마을회관 뒷산,경상북도 포항시 남구 동해면 흥환리 314,129.504148,36.026393,100


In [75]:
pohang.shape

(114, 7)

In [76]:
parking.shape, town.shape, wide_place.shape

((11, 7), (208, 7), (20, 7))

### 4-1-3. 경로당 / 노인정 / 버스

In [77]:
pohang

,시도명,시군구명,시설명,상세주소,경도,위도,수용가능인원수
0,경상북도,포항시,양학생활체육운동장,경상북도 포항시 북구 득량동 219-0,129.340831,36.036154,5362
2,경상북도,포항시,두호동산,경상북도 포항시 북구 두호동 산30-1,129.383434,36.070125,1485
3,경상북도,포항시,동북지방통계청,경상북도 포항시 북구 용흥동 101-4,129.356454,36.034660,1650
4,경상북도,포항시,죽도동행정복지센터,경상북도 포항시 북구 죽도동 71-6,129.363502,36.031127,412
5,경상북도,포항시,죽도빗물펌프장,경상북도 포항시 북구 죽도동 45-1,129.363857,36.033633,578
...,...,...,...,...,...,...,...
348,경상북도,포항시북구,환호 여중 앞,경상북도 포항시 북구 환호동 69,129.400759,36.071086,370
349,경상북도,포항시북구,화진1리 버스정류장 옆,경상북도 포항시 북구 송라면 화진리 261-1,129.382398,36.236579,210
350,경상북도,포항시북구,화진2리 화진비치펜션 앞,경상북도 포항시 북구 송라면 화진리 180-1,129.381916,36.239312,208
351,경상북도,포항시북구,포항해양과학고 앞,경상북도 포항시 북구 환호동 2-8,129.406166,36.074059,200


In [78]:
pohang['시설명'].unique()

array(['양학생활체육운동장', '두호동산', '동북지방통계청', '죽도동행정복지센터', '죽도빗물펌프장',
       '포항대학교 운동장', '포항환경학교 운동장', '성곡농장경로당 주변', '성곡3리 새마을복지관 주변',
       '청진3리 어민복지회관 주변', '호암경로당 주변', '청하면민복지회관', '기일리 노인정 주변',
       '(구)지경리검문소 주변', '기계면사무소', '지가1리 마을숲 주변', '고지리 마을쉼터 주변',
       '광천1리 경로당 주변', '친환경영농지원센터', '포항학생야영장', '농업기술센터', '양덕축구장',
       '방석2리부둣가', '방석보건진료소', '수곡리 경로당 주변', '진전리 경로당 주변', '석병2리 경로당',
       '석병1리경로당', '홍계리경로당 주변', '대송면행정복지센터', '오천읍민운동장', '오천읍행정복지센터',
       '계원1리 공동작업장 주변', '동해면민운동장', '석리 경로당 주변', '청룡회관 주변', '임곡교 옆 삼거리',
       '세계2리 경로당 주변', '금오리 경로당 주변', '발산리 어민복지회관 주변', '장기면사무소',
       '금곡리 노인정 주변', '호미곶해맞이광장', '호미곶면사무소', '(구)포항역 폐철도부지', '구룡포과메기문화관',
       '포항종합운동장', '포항운하부지', '송림테마거리', '포스텍 운동장', '하정1리 선박출입대행신고소 주변',
       '하정3리 버스승강장 주변', '구룡포JC회관', '연일읍행정복지센터', '청림운동장',
       '(구)구남분교장(구룡포초교) 운동장', '벤트풀빌라 옆', '해송모텔 앞', '충혼탑 앞', '해은사 앞',
       '한빛수산 입구 도로', '구만리 389-1 주변', '호미곶팬션 앞', '대동배1리마을뒷산',
       '두원리 버스승강장 앞', '두원주유소 앞', '대진주유소 앞', 'MGM 그랜드모텔 앞', '우리수산 앞',
       '삼

In [79]:
town2 = pohang[pohang['시설명'].str.contains('경로당')]

In [80]:
town3 = pohang[pohang['시설명'].str.contains('노인정')]

In [81]:
bus = pohang[pohang['시설명'].str.contains('버스')]

In [82]:
pohang = pohang.drop(index = town2.index)

In [83]:
pohang = pohang.drop(index = town3.index)

In [84]:
pohang = pohang.drop(index = bus.index)

### 4-1-4. 주유소 운동장

In [85]:
pohang['시설명'].unique()

array(['양학생활체육운동장', '두호동산', '동북지방통계청', '죽도동행정복지센터', '죽도빗물펌프장',
       '포항대학교 운동장', '포항환경학교 운동장', '성곡3리 새마을복지관 주변', '청진3리 어민복지회관 주변',
       '청하면민복지회관', '(구)지경리검문소 주변', '기계면사무소', '지가1리 마을숲 주변', '고지리 마을쉼터 주변',
       '친환경영농지원센터', '포항학생야영장', '농업기술센터', '양덕축구장', '방석2리부둣가', '방석보건진료소',
       '대송면행정복지센터', '오천읍민운동장', '오천읍행정복지센터', '계원1리 공동작업장 주변', '동해면민운동장',
       '청룡회관 주변', '임곡교 옆 삼거리', '발산리 어민복지회관 주변', '장기면사무소', '호미곶해맞이광장',
       '호미곶면사무소', '(구)포항역 폐철도부지', '구룡포과메기문화관', '포항종합운동장', '포항운하부지',
       '송림테마거리', '포스텍 운동장', '하정1리 선박출입대행신고소 주변', '구룡포JC회관', '연일읍행정복지센터',
       '청림운동장', '(구)구남분교장(구룡포초교) 운동장', '벤트풀빌라 옆', '해송모텔 앞', '충혼탑 앞',
       '해은사 앞', '한빛수산 입구 도로', '구만리 389-1 주변', '호미곶팬션 앞', '대동배1리마을뒷산',
       '두원주유소 앞', '대진주유소 앞', 'MGM 그랜드모텔 앞', '우리수산 앞', '세계원 횟집 앞',
       '구룡포청소년회관 앞', '대성수산 입구 앞', '석병교회 옆', '부산돼지국밥 옆', '황제노래연습장 앞',
       '백경오토캠핑장 앞', '청룡회관 입구', '대동배1리보건진료소 앞', '대동배2리마을 뒤', '대보1교차로 앞',
       '호미곶면사무소 앞', '포스코수련원 앞', '1박2일펜션 앞', '방어침례교회 앞', '월포백번횟집앞',
       '포항시립아트홀 앞', '사방기

In [86]:
gas = pohang[pohang['시설명'].str.contains('주유소')]

In [87]:
ground = pohang[pohang['시설명'].str.contains('운동장')]

In [88]:
pohang = pohang.drop(index = gas.index)

In [89]:
pohang = pohang.drop(index = ground.index)

### 4-1-5. 회관

In [90]:
pohang['시설명'].unique()

array(['두호동산', '동북지방통계청', '죽도동행정복지센터', '죽도빗물펌프장', '성곡3리 새마을복지관 주변',
       '청진3리 어민복지회관 주변', '청하면민복지회관', '(구)지경리검문소 주변', '기계면사무소',
       '지가1리 마을숲 주변', '고지리 마을쉼터 주변', '친환경영농지원센터', '포항학생야영장', '농업기술센터',
       '양덕축구장', '방석2리부둣가', '방석보건진료소', '대송면행정복지센터', '오천읍행정복지센터',
       '계원1리 공동작업장 주변', '청룡회관 주변', '임곡교 옆 삼거리', '발산리 어민복지회관 주변', '장기면사무소',
       '호미곶해맞이광장', '호미곶면사무소', '(구)포항역 폐철도부지', '구룡포과메기문화관', '포항운하부지',
       '송림테마거리', '하정1리 선박출입대행신고소 주변', '구룡포JC회관', '연일읍행정복지센터', '벤트풀빌라 옆',
       '해송모텔 앞', '충혼탑 앞', '해은사 앞', '한빛수산 입구 도로', '구만리 389-1 주변',
       '호미곶팬션 앞', '대동배1리마을뒷산', 'MGM 그랜드모텔 앞', '우리수산 앞', '세계원 횟집 앞',
       '구룡포청소년회관 앞', '대성수산 입구 앞', '석병교회 옆', '부산돼지국밥 옆', '황제노래연습장 앞',
       '백경오토캠핑장 앞', '청룡회관 입구', '대동배1리보건진료소 앞', '대동배2리마을 뒤', '대보1교차로 앞',
       '호미곶면사무소 앞', '포스코수련원 앞', '1박2일펜션 앞', '방어침례교회 앞', '월포백번횟집앞',
       '포항시립아트홀 앞', '사방기념관 입구', '오도해수욕장 입구', '오션뷰 펜션 앞', '용한1리 마을입구',
       '칠포해수욕장위부대앞', '해병대 북문 주변', '장길리교회 앞', '하정리 269번지 도로', '월포다리',
       '월포초교 뒷뜰앞', '이가리 산68-3 일원', '이

In [91]:
town4 = pohang[pohang['시설명'].str.contains('회관')]

In [92]:
pohang = pohang.drop(index = town4.index)

### 4-1-6. 센터

In [93]:
pohang['시설명'].unique()

array(['두호동산', '동북지방통계청', '죽도동행정복지센터', '죽도빗물펌프장', '성곡3리 새마을복지관 주변',
       '(구)지경리검문소 주변', '기계면사무소', '지가1리 마을숲 주변', '고지리 마을쉼터 주변',
       '친환경영농지원센터', '포항학생야영장', '농업기술센터', '양덕축구장', '방석2리부둣가', '방석보건진료소',
       '대송면행정복지센터', '오천읍행정복지센터', '계원1리 공동작업장 주변', '임곡교 옆 삼거리', '장기면사무소',
       '호미곶해맞이광장', '호미곶면사무소', '(구)포항역 폐철도부지', '구룡포과메기문화관', '포항운하부지',
       '송림테마거리', '하정1리 선박출입대행신고소 주변', '연일읍행정복지센터', '벤트풀빌라 옆', '해송모텔 앞',
       '충혼탑 앞', '해은사 앞', '한빛수산 입구 도로', '구만리 389-1 주변', '호미곶팬션 앞',
       '대동배1리마을뒷산', 'MGM 그랜드모텔 앞', '우리수산 앞', '세계원 횟집 앞', '대성수산 입구 앞',
       '석병교회 옆', '부산돼지국밥 옆', '황제노래연습장 앞', '백경오토캠핑장 앞', '대동배1리보건진료소 앞',
       '대동배2리마을 뒤', '대보1교차로 앞', '호미곶면사무소 앞', '포스코수련원 앞', '1박2일펜션 앞',
       '방어침례교회 앞', '월포백번횟집앞', '포항시립아트홀 앞', '사방기념관 입구', '오도해수욕장 입구',
       '오션뷰 펜션 앞', '용한1리 마을입구', '칠포해수욕장위부대앞', '해병대 북문 주변', '장길리교회 앞',
       '하정리 269번지 도로', '월포다리', '월포초교 뒷뜰앞', '이가리 산68-3 일원', '이가힐펜션 앞',
       '화진휴게소 입구', '선정마을입구 앞', '칠포1리 고지대', '환호 여중 앞', '화진2리 화진비치펜션 앞',
       '포항해양과학고 앞', '해안로511번

In [94]:
center = pohang[pohang['시설명'].str.contains('센터')]

In [95]:
pohang = pohang.drop(index = center.index)

### 4-1-7. 기타(동산 / ~장 / 삼거리 / 

In [96]:
pohang['시설명'].unique()

array(['두호동산', '동북지방통계청', '죽도빗물펌프장', '성곡3리 새마을복지관 주변', '(구)지경리검문소 주변',
       '기계면사무소', '지가1리 마을숲 주변', '고지리 마을쉼터 주변', '포항학생야영장', '양덕축구장',
       '방석2리부둣가', '방석보건진료소', '계원1리 공동작업장 주변', '임곡교 옆 삼거리', '장기면사무소',
       '호미곶해맞이광장', '호미곶면사무소', '(구)포항역 폐철도부지', '구룡포과메기문화관', '포항운하부지',
       '송림테마거리', '하정1리 선박출입대행신고소 주변', '벤트풀빌라 옆', '해송모텔 앞', '충혼탑 앞',
       '해은사 앞', '한빛수산 입구 도로', '구만리 389-1 주변', '호미곶팬션 앞', '대동배1리마을뒷산',
       'MGM 그랜드모텔 앞', '우리수산 앞', '세계원 횟집 앞', '대성수산 입구 앞', '석병교회 옆',
       '부산돼지국밥 옆', '황제노래연습장 앞', '백경오토캠핑장 앞', '대동배1리보건진료소 앞', '대동배2리마을 뒤',
       '대보1교차로 앞', '호미곶면사무소 앞', '포스코수련원 앞', '1박2일펜션 앞', '방어침례교회 앞',
       '월포백번횟집앞', '포항시립아트홀 앞', '사방기념관 입구', '오도해수욕장 입구', '오션뷰 펜션 앞',
       '용한1리 마을입구', '칠포해수욕장위부대앞', '해병대 북문 주변', '장길리교회 앞', '하정리 269번지 도로',
       '월포다리', '월포초교 뒷뜰앞', '이가리 산68-3 일원', '이가힐펜션 앞', '화진휴게소 입구',
       '선정마을입구 앞', '칠포1리 고지대', '환호 여중 앞', '화진2리 화진비치펜션 앞', '포항해양과학고 앞',
       '해안로511번길10'], dtype=object)

In [97]:
import pandas as pd

In [98]:
pohang =pd.read_csv('포항시 대피소 현황.csv')

In [99]:
pohang.shape

(497, 7)

In [100]:
pohang['시설명'].unique()

array(['양학중학교 운동장', '양학생활체육운동장', '양학초등학교 운동장', '선린요양병원 주차장',
       '(구)포항중앙초등학교 운동장', '포항고등학교 운동장', '포항항도초등학교 운동장', '포항여자고등학교 운동장',
       '포항중학교 운동장', '포항초등학교 운동장', '포항장흥초등학교 운동장', '장성초등학교 운동장',
       '장량초등학교 운동장', '장성4어린이공원', '장성6어린이공원', '장성15어린이공원', '두호5어린이공원',
       '두호동산', '물의공원', '창포6공원', '두호1어린이공원', '두호고등학교 운동장', '장원초등학교 운동장',
       '유성여자고등학교 운동장', '창포중학교 운동장', '두호초등학교 운동장', '창포초등학교 운동장',
       '포항동부초등학교 운동장', '두호남부초등학교 운동장', '경북과학고등학교 운동장', '동북지방통계청',
       '포항여자중학교 운동장', '세화고등학교 운동장', '대동중고등학교 운동장', '중앙초등학교 운동장',
       '포항대흥초등학교 운동장', '동지여자고등학교 운동장', '대흥중학교 운동장', '용흥중학교 운동장',
       '포항남부초등학교 운동장', '포항여자전자고등학교 운동장', '죽도동행정복지센터', '죽도빗물펌프장',
       '죽도초등학교 운동장', '대잠초등학교 운동장', '죽도2어린이공원', '용흥초등학교 운동장',
       '덕장1리 마을회관 주변', '용전2리 마을회관 주변', '용전1리 마을회관 주변', '곡강초등학교 운동장',
       '영흥초등학교 운동장', '포항항도중학교 운동장', '용곡리 마을회관 주변', '양백1리 마을회관 주변',
       '양백2리 마을회관 주변', '덕성2리 마을회관 주변', '덕성1리 마을회관 주변', '덕장2리 마을회관 주변',
       '용한1리 마을회관 주변', '죽천초등학교 운동장', '죽천2리 마을회관 주변', '죽천1리 마을회관 주변',

# 5. 분석단계

## 5-1. 행정동 추리기

In [101]:
popo['수용인원수합'].sum()/popo['유동인구수'].sum() * 100
#포항시전체의 유동인구수 대비 수용인원비율?

66.15389264166224

In [102]:
# 갭 차이가 많이 난다. -> 수용인원비율 100 이상인 곳을 살펴보니 시골의 넓은 부지, 적은 유동인구로 인해 비율이 높게 잡힌 것을 확인 하였다.
popo_total.sort_values('유동인구수 대비 수용인원비율',ascending = False).head()

,구,행정동명,유동인구수,수용인원수합,유동인구수 대비 수용인원비율,2019년08월_총인구수,총인구수 대비 수용인원비율
0,북구,신광면,1660.226,5765,347.241882,2882,200.034698
1,남구,장기면,4053.387,13958,344.353993,4291,325.285481
2,북구,환여동,5821.419,15780,271.067930,10000,157.800000
3,북구,기북면,648.323,1338,206.378611,1311,102.059497
4,북구,죽장면,2421.097,4068,168.023008,2174,187.120515


In [103]:
# 유동인구수 대비 수용인원비율이 100이 아니라 66 평균보다 더 낮은 것.
ui=['신광면', '장기면', '환여동', '기북면', '죽장면', '호미곶면', '구룡포읍', '양학동', '송라면', '해도동', '송도동', '대송면']

In [104]:
yu = pd.DataFrame(columns=['Unnamed: 0', '시도명', '시군구명', '시설명', '상세주소', '경도', '위도', '수용가능인원수'])

In [105]:
data = []
for s in ui:
    temp = []
    path = f'./포항시/{s}대피소.csv'
    df1 = pd.read_csv(path,encoding = 'euckr')
    temp.append(s)
    for i in range(len(df1)):
        data.append(temp)
    yu = pd.concat([yu,df1])

In [106]:
wow =pd.DataFrame(data)
yu = yu.reset_index(drop= 'index')
yu = pd.concat([yu,wow],axis = 1)

In [107]:
#yu에 인구랑 유동인구수 넣기
rt = popo[['행정동명','유동인구수']]
yu = pd.merge(yu,roro,left_on=0,right_on='행정동명')
yu = pd.merge(yu,rt,on='행정동명')

In [108]:
#유동인구수 대비 수용인원비율이 100이 넘어가는 행정동의 대피소들
yu

,Unnamed: 0,시도명,시군구명,시설명,상세주소,경도,위도,수용가능인원수,0,2019년08월_총인구수,행정동명,유동인구수
0,2117,경상북도,포항시,신광중학교 운동장,경상북도 포항시 북구 신광면 토성리 344-0,129.263928,36.130463,1818,신광면,"2,882",신광면,1660.226
1,2118,경상북도,포항시,사정1리 마을회관 주변,경상북도 포항시 북구 신광면 사정리 928-1,129.271198,36.132922,61,신광면,"2,882",신광면,1660.226
2,2119,경상북도,포항시,사정2리 마을회관 주변,경상북도 포항시 북구 신광면 사정리 501-2,129.274433,36.141739,61,신광면,"2,882",신광면,1660.226
3,2120,경상북도,포항시,안덕1리 마을회관 주변,경상북도 포항시 북구 신광면 안덕리 233-0,129.26664,36.14613,61,신광면,"2,882",신광면,1660.226
4,2121,경상북도,포항시,안덕2리 마을회관 주변,경상북도 포항시 북구 신광면 안덕리 562-4,129.266742,36.158083,61,신광면,"2,882",신광면,1660.226
...,...,...,...,...,...,...,...,...,...,...,...,...
214,6588,경상북도,포항시,대송면행정복지센터,경상북도 포항시 남구 대송면 송동리 782-1,129.360284,35.968857,909,대송면,"3,689",대송면,5869.581
215,6589,경상북도,포항시,대송초등학교 운동장,경상북도 포항시 남구 대송면 제내리 974-0,129.362424,35.984352,1818,대송면,"3,689",대송면,5869.581
216,6590,경상북도,포항시,남성1리 마을회관 주변,경상북도 포항시 남구 대송면 남성리 268-1,129.353454,35.976211,61,대송면,"3,689",대송면,5869.581
217,6614,경상북도,포항시,대송중학교 운동장,경상북도 포항시 남구 대송면 남성리 83-5,129.359599,35.97774,1515,대송면,"3,689",대송면,5869.581


In [109]:
#ty 제외한 나머지 부분을 살펴 봐도 인구에 비해 부지가 너무 넓다.
ty=['마을회관','행정','학교','노인정','경로당','주차장','공터']
a=0
tr=[]
for i in ty:
    a+=len(yu[yu['시설명'].str.contains(i)])
    tr+=list(yu[yu['시설명'].str.contains(i)].index)

In [110]:
# 마을회관/행정/학교/노인정/경로당/주차장/공터의 개수가 반이 넘어가는 것을 확인 할 수 있다.
a

159

In [111]:
yu2 = yu.drop(index=tr)

In [112]:
#ty 제외한 나머지 부분을 살펴 봐도 인구에 비해 부지가 너무 넓다라는 것을 알 수 있다.
yu2.head(10)

,Unnamed: 0,시도명,시군구명,시설명,상세주소,경도,위도,수용가능인원수,0,2019년08월_총인구수,행정동명,유동인구수
29,6612,경상북도,포항시,계원1리 공동작업장 주변,경상북도 포항시 남구 장기면 계원리 96-0,129.528588,35.870607,61,장기면,"4,291",장기면,4053.387
49,6691,경상북도,포항시,장기면사무소,경상북도 포항시 남구 장기면 읍내리 108-0,129.492784,35.895964,303,장기면,"4,291",장기면,4053.387
56,11177,경상북도,포항시 남구,두원리 버스승강장 앞,경상북도 포항시 남구 장기면 두원리 523-15,129.515828,35.84356,100,장기면,"4,291",장기면,4053.387
57,11178,경상북도,포항시 남구,두원주유소 앞,경상북도 포항시 남구 장기면 두원리 57-1,129.518114,35.84891,1500,장기면,"4,291",장기면,4053.387
58,11179,경상북도,포항시 남구,대진주유소 앞,경상북도 포항시 남구 장기면 대진리 87-3,129.517354,35.923007,58,장기면,"4,291",장기면,4053.387
60,11188,경상북도,포항시 남구,세계원 횟집 앞,경상북도 포항시 남구 장기면 계원리 324,129.525328,35.861501,1011,장기면,"4,291",장기면,4053.387
61,11189,경상북도,포항시 남구,등대회식당 버스승강장,경상북도 포항시 남구 장기면 계원리 26-3,129.530046,35.868253,1500,장기면,"4,291",장기면,4053.387
62,11194,경상북도,포항시 남구,부산돼지국밥 옆,경상북도 포항시남구 장기면 신창리 237-2,129.523232,35.892418,70,장기면,"4,291",장기면,4053.387
64,11196,경상북도,포항시 남구,황제노래연습장 앞,경상북도 포항시남구 장기면 신창리 479-4,129.526652,35.885226,97,장기면,"4,291",장기면,4053.387
67,11199,경상북도,포항시 남구,영암리 버스승강장 앞,경상북도 포항시남구 장기면 영암리 101-2,129.525999,35.904067,265,장기면,"4,291",장기면,4053.387


In [113]:
# 따라서 포항시 유동인구수 비율이 100이하 인 곳을 살펴보도록 하자.

## 5-2. 포항시 유동인구수 비율이 100이하

In [114]:
popo

,구,행정동명,유동인구수,수용인원수합,유동인구수 대비 수용인원비율
27,북구,신광면,1660.226,5765,347.241882
23,남구,장기면,4053.387,13958,344.353993
22,북구,환여동,5821.419,15780,271.067930
28,북구,기북면,648.323,1338,206.378611
26,북구,죽장면,2421.097,4068,168.023008
25,남구,호미곶면,3357.581,5056,150.584602
18,남구,구룡포읍,7378.871,10902,147.746180
24,북구,양학동,3427.968,4515,131.710681
19,북구,송라면,5901.258,6944,117.669826
12,남구,해도동,12714.806,14243,112.019012


In [115]:
# 100보다 작은거 추리기
popo2 = popo[popo['유동인구수 대비 수용인원비율'] < 100]

In [116]:
#100보다 작은 지역의 비율
popo2['수용인원수합'].sum() /popo2['유동인구수'].sum() * 100

51.797486754846325

In [117]:
#포항시 전체 비율인 66.15389264166224 보다 더 떨어진 것을 확인 가능.

In [118]:
#프로젝트 목적이 
popo3 = popo[popo['유동인구수 대비 수용인원비율'] < 51.797486754846325]

In [119]:
popo3

,구,행정동명,유동인구수,수용인원수합,유동인구수 대비 수용인원비율
7,남구,연일읍,27537.452,13611,49.427231
1,북구,장량동,41794.774,19820,47.422197
15,남구,청림동,9629.903,4438,46.085615
6,북구,우창동,27700.710,11818,42.663166
2,북구,중앙동,37537.677,12967,34.543960
5,남구,대이동,28592.226,7788,27.238173
4,북구,죽도동,30559.290,7698,25.190376
8,남구,제철동,21918.613,5151,23.500575


## 5-3. 최종 대피소 추천

In [120]:
#최종적으로 마을회관 / 공원 등 대피소 추린 장소.
tot = pd.read_csv('지진대피소 후보지v2. 31곳 (2).csv', encoding = 'euckr')

In [121]:
# popo3 이랑 동일 한 것 비교
tot['행정동'].unique()

array(['연일읍', '장량동', '청림동', '우창동', '대이동', '죽도동', '제철동', '중앙동'],
      dtype=object)

In [122]:
tot.head()

,행정동,시설명,도로명 주소,지번주소,규모(면적㎡),대피시설 수용인원(명)
0,연일읍,포항경북자동차학원,경북 포항시 남구 연일읍 연일로 143,경북 포항시 남구 연일읍 생지리 136-2,8819,2672.424242
1,연일읍,유강2공원,NaN,연일읍 유강리 601-2,3009,911.818182
2,연일읍,연일성당 주차장,경북 포항시 남구 연일읍 형산강남로326번길 22,경상북도 포항시 남구 연일읍 생지리 143,1435,434.848485
3,연일읍,포항식자재마트 앞 공터,경북 포항시 남구 연일읍 연일로159번길 6,경상북도 포항시 남구 연일읍 생지리 327-1,990,300.000000
4,장량동,장성풋살구장,NaN,장성동 1504-4,3436,1041.212121


In [123]:
# NAN값 제거
tot = tot.drop(index =25)

In [124]:
final = pd.merge(popo3,tot[['행정동','시설명','대피시설 수용인원(명)']],left_on='행정동명',right_on='행정동')

In [125]:
final['대피시설 수용인원(명)'] = round(final['대피시설 수용인원(명)']).astype(int)

In [126]:
final = final[['구','행정동명','유동인구수','시설명','대피시설 수용인원(명)']]

In [127]:
final.head()

,구,행정동명,유동인구수,시설명,대피시설 수용인원(명)
0,남구,연일읍,27537.452,포항경북자동차학원,2672
1,남구,연일읍,27537.452,유강2공원,912
2,남구,연일읍,27537.452,연일성당 주차장,435
3,남구,연일읍,27537.452,포항식자재마트 앞 공터,300
4,북구,장량동,41794.774,장성풋살구장,1041


In [128]:
final2 = final.groupby('행정동명',as_index=False)['대피시설 수용인원(명)'].sum()

In [129]:
final2

,행정동명,대피시설 수용인원(명)
0,대이동,5631
1,연일읍,4319
2,우창동,4290
3,장량동,41844
4,제철동,6255
5,죽도동,1360
6,중앙동,3009
7,청림동,1632


In [130]:
final2['행정동명'].unique()

array(['대이동', '연일읍', '우창동', '장량동', '제철동', '죽도동', '중앙동', '청림동'],
      dtype=object)

In [131]:
import numpy as np

In [132]:
# popo2 -> 유동인구 비율이 100이하인 지역
# 기존 수용인원수랑 합치는 과정.
for j,i in enumerate(final2['행정동명'].unique()):
    popo2.loc[popo2['행정동명']==i,'수용인원수합2'] = final2.loc[j,'대피시설 수용인원(명)'] + popo2.loc[popo2['행정동명']==i,'수용인원수합']

C:\Users\user\AppData\Local\Temp\ipykernel_1724\778485112.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  popo2.loc[popo2['행정동명']==i,'수용인원수합2'] = final2.loc[j,'대피시설 수용인원(명)'] + popo2.loc[popo2['행정동명']==i,'수용인원수합']


In [135]:
popo2['수용인원수합2'] = popo2['수용인원수합2'].fillna(popo2['수용인원수합'])

C:\Users\user\AppData\Local\Temp\ipykernel_1724\565185220.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  popo2['수용인원수합2'] = popo2['수용인원수합2'].fillna(popo2['수용인원수합'])


In [137]:
popo2

,구,행정동명,유동인구수,수용인원수합,유동인구수 대비 수용인원비율,수용인원수합2
11,남구,용흥동,14072.677,13333,94.743879,13333.0
16,남구,동해면,9360.774,8115,86.691549,8115.0
10,남구,효곡동,17587.516,13787,78.390831,13787.0
20,북구,기계면,5884.323,4191,71.223147,4191.0
3,남구,오천읍,34419.903,24398,70.883407,24398.0
14,북구,청하면,10661.613,7336,68.807600,7336.0
13,북구,두호동,11494.581,7546,65.648326,7546.0
0,북구,흥해읍,48547.871,31787,65.475580,31787.0
9,남구,상대동,19402.742,11698,60.290448,11698.0
7,남구,연일읍,27537.452,13611,49.427231,17930.0


In [140]:
popo2['수용인원수합'].sum() /popo2['유동인구수'].sum() * 100

51.797486754846325

In [141]:
# 기존 비율보다 올라간 것을 확인 가능.
popo2['수용인원수합2'].sum() /popo2['유동인구수'].sum() * 100

69.02449566475667

In [142]:
popo2['유동인구수 대비 수용인원비율2'] = popo2['수용인원수합2'] / popo2['유동인구수'] * 100

C:\Users\user\AppData\Local\Temp\ipykernel_1724\2482332275.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  popo2['유동인구수 대비 수용인원비율2'] = popo2['수용인원수합2'] / popo2['유동인구수'] * 100


In [143]:
popo2

,구,행정동명,유동인구수,수용인원수합,유동인구수 대비 수용인원비율,수용인원수합2,유동인구수 대비 수용인원비율2
11,남구,용흥동,14072.677,13333,94.743879,13333.0,94.743879
16,남구,동해면,9360.774,8115,86.691549,8115.0,86.691549
10,남구,효곡동,17587.516,13787,78.390831,13787.0,78.390831
20,북구,기계면,5884.323,4191,71.223147,4191.0,71.223147
3,남구,오천읍,34419.903,24398,70.883407,24398.0,70.883407
14,북구,청하면,10661.613,7336,68.807600,7336.0,68.807600
13,북구,두호동,11494.581,7546,65.648326,7546.0,65.648326
0,북구,흥해읍,48547.871,31787,65.475580,31787.0,65.475580
9,남구,상대동,19402.742,11698,60.290448,11698.0,60.290448
7,남구,연일읍,27537.452,13611,49.427231,17930.0,65.111325


In [144]:
# popo2.to_csv('포항대피소최종본.csv',encoding='euckr',index = False)